## Using ak_requests

In [3]:
import common as ak
import materials, craft, stage
import numpy as np
from scipy.optimize import linprog

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
item_dict = ak.get_item_dict(lang="en_US", local=True)
stage_dict = ak.get_stage_dict(lang="zh_CN", local=True)
craft_dict = ak.get_craft_dict(lang="zh_CN", local=True)
psdf = ak.get_pengstats_df(server="US")

Reading from local file
Reading from remote
Reading from local file


In [4]:
stage_san_cost, stage_names, stage_names_rev, stage_lmd_dict = stage.get_stage_info(psdf, stage_dict)
item_names, item_names_rev = ak.get_item_info(item_dict)
event_names, event_names_rev = stage.get_event_info(stage_names)

In [5]:
material_ids = ak.get_material_ids(item_names_rev, materials.MATERIAL_NAMES)
n_mats_all = len(material_ids)

In [7]:
#main_stage_ids = ak.get_main_stage_ids(psdf)
main_stage_ids = stage.get_main_and_perm_stage_ids(psdf)
stage_ids_all = np.concatenate((main_stage_ids,[
    "wk_melee_5",
    "wk_kc_5",
    "wk_fly_5"
]))

stage_san_all = stage.get_san_cost(stage_ids_all, stage_san_cost)
n_stages_all = len(stage_ids_all)

In [8]:
craft_matrix, subprod_matrix = craft.get_craft_matrix(craft_dict, material_ids, item_names_rev)

In [9]:
drop_matrix_all, coeff_matrix_all = stage.get_drop_matrix(psdf, stage_ids_all, material_ids, stage_lmd_dict)

In [11]:
drop_matrix, stage_san, stage_ids = stage.filter_drop_matrix(drop_matrix_all, stage_san_all, stage_ids_all)

In [12]:
cutoff_threshold = 0.01
drop_matrix = stage.drop_matrix_cutoff(drop_matrix, cutoff_threshold)

In [13]:
byprod_rate = 0.18
craft_constraint_matrix = ak.get_craft_constraint_matrix(craft_matrix, subprod_matrix, byprod_rate)

In [14]:
coeff = np.sum(drop_matrix, axis=0)
res = linprog(-coeff,
              A_ub=drop_matrix, b_ub=stage_san,
              A_eq=craft_constraint_matrix, b_eq=np.zeros(len(craft_constraint_matrix)),
              bounds=(0, None)
             )

print(res.success)
## Make dict from itemId -> msv at the end
msv = res.x
msv_dict = {}
for i, d in enumerate(material_ids):
    msv_dict[d] = msv[i]
    print("{}: {:.4f}".format(item_names[d], msv[i]))

True
LMD: 0.0040
Skill Summary - 1: 1.5618
Skill Summary - 2: 3.9706
Skill Summary - 3: 10.0946
Orirock: 1.3317
Orirock Cube: 4.0524
Orirock Cluster: 20.0232
Orirock Concentration: 76.6195
Damaged Device: 2.3536
Device: 7.1182
Integrated Device: 28.2339
Optimized Device: 93.8480
Ester: 1.8342
Polyester: 5.5599
Polyester Pack: 22.0004
Polyester Lump: 92.1510
Sugar Substitute: 1.7557
Sugar: 5.3243
Sugar Pack: 21.0582
Sugar Lump: 96.5974
Oriron Shard: 2.4794
Oriron: 7.4954
Oriron Cluster: 29.7426
Oriron Block: 106.2463
Diketon: 2.3610
Polyketon: 7.1404
Aketon: 28.3226
Keton Colloid: 102.4417
Loxic Kohl: 23.3009
White Horse Kohl: 76.6213
Manganese Ore: 28.2117
Manganese Trihydrate: 98.2515
Grindstone: 29.0410
Grindstone Pentahydrate: 83.5442
RMA70-12: 35.7355
RMA70-24: 100.6311
Polymerization Preparation: 270.8765
Bipolar Nanoflake: 232.6596
D32 Steel: 267.9959
Coagulating Gel: 32.2934
Polymerized Gel: 85.9626
Incandescent Alloy: 27.3999
Incandescent Alloy Block: 81.2015
Crystalline Compon

In [15]:
craft.print_craft_materials("Cutting Fluid Solution", craft_matrix, material_ids, item_names, item_names_rev)

LMD: 300
RMA70-12: 1
Crystalline Component: 1
Compound Cutting Fluid: 1


In [16]:
stage.print_stage_drops("R8-11", drop_matrix, stage_ids, material_ids, stage_names_rev, item_names)

LMD: 252
Crystalline Component: 49.12%
Orirock Cube: 35.25%
Orirock: 20.58%
Device: 15.27%
Damaged Device: 7.56%
Orirock Cluster: 4.16%
Loxic Kohl: 2.95%
Coagulating Gel: 2.45%
Crystalline Circuit: 2.20%
Integrated Device: 1.65%


In [ ]:
event_ids = stage.get_event_ids("BI", psdf, event_names_rev, remove_permanent=True)
event_stage_san = stage.get_san_cost(event_ids, stage_san_cost)
event_drop_matrix, _ = ak.get_drop_matrix(psdf, event_ids, material_ids, stage_lmd_dict)
main_drops = ak.get_main_drops(event_drop_matrix, msv, material_ids)
#sanity_return = np.matmul(event_drop_matrix, msv)
stack = np.stack([event_ids, ak.get_stage_efficiency(event_drop_matrix, msv, event_stage_san), main_drops]).T
for a, b, c in stack:
    print("{}: {:.4} {}".format(stage_names[a], float(b), item_names[c]))

In [ ]:
item_idx = np.where(material_ids == item_names_rev["Compound Cutting Fluid"])[0][0]
drops = drop_matrix_all.T[:][item_idx]
print(len(drops), len(material_ids), len(stage_ids), drop_matrix_all.shape)
for i, d in enumerate(drops):
    if d > 0.005:
        print("{}: {:.4f}%".format(stage_names[stage_ids[i]], 100*d))